Decomposing wide table into BCNF. 

In [1]:
import pandas as pd
import os
os.getcwd()


'/'

In [2]:

from pathlib import Path
import os
import subprocess

# Set any folder you know is in the repo
start = Path("/Users/ellawileman/Documents/fall2025/db_management/final_project_database/final_project_database")

# Walk upward until .git is found
current = start
while True:
    if ".git" in os.listdir(current):
        repo_root = current
        break
    parent = current.parent
    if parent == current:
        raise RuntimeError("Not inside a Git repo")
    current = parent

os.chdir(repo_root)
print("Repo root:", repo_root)

subprocess.run(["git", "status"]) 



Repo root: /Users/ellawileman/Documents/fall2025/db_management/final_project_database
On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.DS_Store
	final_project_database/.DS_Store
	final_project_database/p2_starter_code/.DS_Store
	final_project_database/p2_starter_code/src/.DS_Store
	final_project_database/p2_starter_code/src/main/.DS_Store
	final_project_database/p2_starter_code/src/main/resources/.DS_Store

nothing added to commit but untracked files present (use "git add" to track)


CompletedProcess(args=['git', 'status'], returncode=0)

In [3]:

import subprocess

# Status
subprocess.run(["git", "status"])

# Example: list branches
subprocess.run(["git", "branch"])

# Pull latest changes
subprocess.run(["git", "pull"])



On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.DS_Store
	final_project_database/.DS_Store
	final_project_database/p2_starter_code/.DS_Store
	final_project_database/p2_starter_code/src/.DS_Store
	final_project_database/p2_starter_code/src/main/.DS_Store
	final_project_database/p2_starter_code/src/main/resources/.DS_Store

nothing added to commit but untracked files present (use "git add" to track)
* main
Already up to date.


CompletedProcess(args=['git', 'pull'], returncode=0)

In [137]:
"""
import os

# List all files/folders in current directory
print(os.listdir())

# Or list them nicely, one per line
for f in os.listdir():
    print(f)
    

# Path to a file inside the repo
file_path = os.path.join(os.getcwd(), "final_project_database", "wide_book_data.csv")

"""

'\nimport os\n\n# List all files/folders in current directory\nprint(os.listdir())\n\n# Or list them nicely, one per line\nfor f in os.listdir():\n    print(f)\n    \n\n# Path to a file inside the repo\nfile_path = os.path.join(os.getcwd(), "final_project_database", "wide_book_data.csv")\n\n'

In [4]:
wide_dat = pd.read_csv("final_project_database/datasource_csvs/test_wide_with_pages.csv")
wide_dat["isbn13"] = wide_dat["isbn13"].astype("int64")

# Standardize column names with schema
wide_dat = wide_dat.rename(columns={"book_id": "bookId", "user_id": "userId", "FirstName": "firstName", "LastName": "lastName", "Username": "username"})

In [5]:
wide_dat = wide_dat.drop(['_merge', 'on_user_list'], axis=1) 
wide_dat.head()

,isbn13,description,genres,bookId,title,authors,average_rating,original_publication_year,ratings_count,image_url,userId,rating,firstName,lastName,total_copies,username,page_count
0,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,1136.0,4.0,George,Roberts,4,groberts,103
1,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,1350.0,4.0,Mario,Hardin,4,mhardin,103
2,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,2276.0,4.0,Michael,Maldonado,4,mmaldonado,103
3,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,2552.0,4.0,Corey,Johnson,4,cjohnson,103
4,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,2996.0,3.0,Samuel,Dalton,4,sdalton,103


In [6]:
wide_dat.dtypes

isbn13                         int64
description                   object
genres                        object
bookId                         int64
title                         object
authors                       object
average_rating               float64
original_publication_year      int64
ratings_count                  int64
image_url                     object
userId                       float64
rating                       float64
firstName                     object
lastName                      object
total_copies                   int64
username                      object
page_count                     int64
dtype: object

R(isbn13, description, genres, book_id, title, authors, average_rating, original_publication_year, ratings_count, image_url, user_id, rating, FirstName, LastName, username, page_count, total_copies)

In [7]:
R = wide_dat

Minimal superkey (candidate key) = (book_id, user_id)

Decompose on FD #1: user_id -> FirstName, LastName, Username

In [8]:
R1 = R[["userId", "firstName", "lastName", "username"]].drop_duplicates().reset_index(drop=True)
R1 = R1.dropna(subset=['userId'])

In [9]:
duplicate_user_ids = R1["userId"].duplicated().sum()
print("Duplicate user_ids:", duplicate_user_ids)


Duplicate user_ids: 0


In [10]:
R1.head()

,userId,firstName,lastName,username
0,1136.0,George,Roberts,groberts
1,1350.0,Mario,Hardin,mhardin
2,2276.0,Michael,Maldonado,mmaldonado
3,2552.0,Corey,Johnson,cjohnson
4,2996.0,Samuel,Dalton,sdalton


In [11]:
R2 = R[["userId", "isbn13", "description", "genres", "bookId", "title", "authors", "average_rating", "original_publication_year", "ratings_count", "image_url", "rating", "total_copies", "page_count"]].drop_duplicates().reset_index(drop=True)
R2 = R2.dropna(subset=['userId'])
R2 = R2.dropna(subset=["bookId"])


In [12]:
R2.head()

,userId,isbn13,description,genres,bookId,title,authors,average_rating,original_publication_year,ratings_count,image_url,rating,total_copies,page_count
0,1136.0,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,4.0,4,103
1,1350.0,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,4.0,4,103
2,2276.0,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,4.0,4,103
3,2552.0,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,4.0,4,103
4,2996.0,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",890,Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,3.0,4,103


Does FD #2 satisfy BCNF? 
No because book_id is not superkey. Decompose.

In [13]:
R3 = R2[["bookId", "isbn13", "description", "genres", "title", "authors", "average_rating", "original_publication_year", "ratings_count", "image_url", "total_copies", "page_count"]].drop_duplicates().reset_index(drop=True)
R3 = R3.dropna(subset=["bookId"])
R3.head()

,bookId,isbn13,description,genres,title,authors,average_rating,original_publication_year,ratings_count,image_url,total_copies,page_count
0,890,9780142000670,The compelling story of two outsiders striving...,"['Classics', 'Fiction', 'School', 'Historical ...",Of Mice and Men,John Steinbeck,3.84,1937,1467496,https://images.gr-assets.com/books/1437235233m...,4,103
1,1953,9780141439600,After eighteen years as a political prisoner i...,"['Classics', 'Fiction', 'Historical Fiction', ...",A Tale of Two Cities,"Charles Dickens, Richard Maxwell, Hablot Knigh...",3.81,1859,637412,https://images.gr-assets.com/books/1344922523m...,3,489
2,3636,9780385732550,Twelve-year-old Jonas lives in a seemingly ide...,"['Young Adult', 'Fiction', 'Classics', 'Dystop...","The Giver (The Giver, #1)",Lois Lowry,4.12,1993,1296825,https://images.gr-assets.com/books/1342493368m...,3,208
3,3836,9780142437230,Don Quixote has become so entranced by reading...,"['Classics', 'Fiction', 'Literature', 'Spanish...",Don Quixote,"Miguel de Cervantes Saavedra, Roberto González...",3.85,1605,134931,https://images.gr-assets.com/books/1364958765m...,1,1023
4,2165,9780684830490,Librarian's note: An alternate cover edition c...,"['Classics', 'Fiction', 'Literature', 'Novels'...",The Old Man and the Sea,Ernest Hemingway,3.73,1952,520630,https://images.gr-assets.com/books/1329189714m...,3,96


In [14]:
R4 = R2[["userId", "bookId", "rating"]].drop_duplicates(subset=["userId", "bookId"], keep="first").reset_index(drop=True)
R4 = R4.dropna(subset=['userId'])
R4 = R4.dropna(subset=["bookId"])
R4.head()

,userId,bookId,rating
0,1136.0,890,4.0
1,1350.0,890,4.0
2,2276.0,890,4.0
3,2552.0,890,4.0
4,2996.0,890,3.0


FD #3 satisfies BCNF so all decomposed relations are now in BCNF. 

In [15]:
# Adding a password column with randomly generated passwords 
import string
import random

# Function to generate a random password
""""
def generate_password(length=10):
    chars = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(chars) for _ in range(length))

# Add a password column
R1['password'] = [generate_password() for _ in range(len(R1))]
"""
R1["password"] = "password123"  # Placeholder password for all users
R1.head()

,userId,firstName,lastName,username,password
0,1136.0,George,Roberts,groberts,password123
1,1350.0,Mario,Hardin,mhardin,password123
2,2276.0,Michael,Maldonado,mmaldonado,password123
3,2552.0,Corey,Johnson,cjohnson,password123
4,2996.0,Samuel,Dalton,sdalton,password123


In [16]:
# Looking for duplicate usernames 
R1["username"].value_counts().head()
duplicate_usernames = R1[R1.duplicated(subset="username", keep=False)]

print(duplicate_usernames)
print("Number of duplicate usernames:", duplicate_usernames["username"].nunique())

Empty DataFrame
Columns: [userId, firstName, lastName, username, password]
Index: []
Number of duplicate usernames: 0


In [17]:
# Make usernames unique to uphold schema constraint
counts = {}
new_usernames = []

for u in R1["username"]:
    if u not in counts:
        counts[u] = 0
        new_usernames.append(u)
    else:
        counts[u] += 1
        new_usernames.append(f"{u}_{counts[u]}")

R1["username"] = new_usernames

In [151]:
!pip install --upgrade pip
!pip install bcrypt
!pip install swifter



In [18]:
import bcrypt
import swifter 

# Ensure passwords are strings and stripped
R1['password'] = R1['password'].astype(str).str.strip()

# Define hashing function
def hash_password(pw: str) -> str:
    pw_bytes = pw[:72].encode('utf-8')  # truncate to 72 bytes
    hashed_bytes = bcrypt.hashpw(pw_bytes, bcrypt.gensalt(rounds=8))  # lower rounds for speed
    return hashed_bytes.decode('utf-8')

# Hash all passwords in parallel
R1['password'] = R1['password'].swifter.apply(hash_password)

# Check

print(R1.head())

Pandas Apply:   0%|          | 0/7089 [00:00<?, ?it/s]

   userId firstName   lastName    username  \
0  1136.0    George    Roberts    groberts   
1  1350.0     Mario     Hardin     mhardin   
2  2276.0   Michael  Maldonado  mmaldonado   
3  2552.0     Corey    Johnson    cjohnson   
4  2996.0    Samuel     Dalton     sdalton   

                                            password  
0  $2b$08$Cv67FGVqGLAxr28ftoS0VuNAXj/QI8XyHQbTjBk...  
1  $2b$08$FkgU5WRTZnh4NEIIzss0D.vGnWcafWZ1rMxtvKz...  
2  $2b$08$Kn5fYco8ZdF57D/D5jzHbOsxvAjNWq3dSf0yIQS...  
3  $2b$08$MxoIAuRLIJytd5KIbj93MurWiji/mTBBeboh6EQ...  
4  $2b$08$8EWoyrrA.mjswC40IscUuOQn.9tIQWBzATVWray...  


In [19]:
import bcrypt

pw = "mypassword123"
hashed = bcrypt.hashpw(pw.encode('utf-8'), bcrypt.gensalt(rounds=10))
print(hashed.decode('utf-8'))


$2b$10$9BVRzdrJJ5fp5igYnpOd2uVk2m1NdaqwGRj8qjCu8khzLc.3hAVSK


In [20]:
# saving the csvs

# Users Table
R1.to_csv("final_project_database/datasource_csvs/users.csv", index = False)

# Books Table
R3.to_csv("final_project_database/datasource_csvs/books.csv", index = False)

# Ratings Table 
R4.to_csv("final_project_database/datasource_csvs/ratings.csv", index = False) 

In [21]:
# Books Table
R3.to_csv("final_project_database/datasource_csvs/books.csv", index = False)

In [22]:
# only keep the entries from to_read table that have valid userId and bookId in the users and books tables

to_read = pd.read_csv("final_project_database/datasource_csvs/to_read.csv")
to_read = to_read.merge(R1[["userId"]], on="userId", how="inner")
to_read = to_read.merge(R3[["bookId"]], on="bookId", how="inner")
to_read.shape

(818, 2)

In [23]:
to_read.to_csv("final_project_database/datasource_csvs/to_read.csv", index = False)